# Query & search registries

This guide walks through all the ways of finding metadata records in LaminDB registries.

In [ ]:
# !pip install lamindb
!lamin init --storage ./test-registries

We'll need some toy data.

In [ ]:
import lamindb as ln

# create toy data
ln.Artifact(ln.core.datasets.file_jpg_paradisi05(), description="My image").save()
ln.Artifact.from_df(ln.core.datasets.df_iris(), description="The iris collection").save()
ln.Artifact(ln.core.datasets.file_fastq(), description="My fastq").save()

# see the content of the artifact registry
ln.Artifact.df()

## Look up metadata

For registries with less than 100k records, auto-completing a `Lookup` object is the most convenient way of finding a record.

For example, take the `User` registry:

In [ ]:
# query the database for all users, optionally pass the field that creates the key
users = ln.User.lookup(field="handle")

# the lookup object is a NamedTuple
users

With auto-complete, we find a specific user record:

In [ ]:
user = users.testuser1
user

You can also get a dictionary:

In [ ]:
users_dict = ln.User.lookup().dict()
users_dict

## Query exactly one record

{class}`~lamindb.core.Record.get` errors if more than one matching records are found.

In [ ]:
# by the universal base62 uid
ln.User.get("DzTjkKse")

# by any expression involving fields
ln.User.get(handle="testuser1")

## Query sets of records

Filter for all artifacts created by a user:

In [ ]:
ln.Artifact.filter(created_by=user).df()

To access the results encoded in a filter statement, execute its return value with one of:

- `.df()`: A pandas `DataFrame` with each record in a row.
- `.all()`: A {class}`~lamindb.core.QuerySet`.
- `.one()`: Exactly one record. Will raise an error if there is none. Is equivalent to the `.get()` method shown above.
- `.one_or_none()`: Either one record or `None` if there is no query result.

```{note}

{meth}`~lamindb.core.Record.filter` returns a {class}`~lamindb.core.QuerySet`.

The ORMs in LaminDB are Django Models and any [Django query](https://docs.djangoproject.com/en/stable/topics/db/queries/) works. LaminDB extends Django's API for data scientists.

Under the hood, any `.filter()` call translates into a SQL select statement.

`.one()` and `.one_or_none()` are two parts of LaminDB's API that are borrowed from SQLAlchemy.

```

## Search for records

Search the toy data:

In [ ]:
ln.Artifact.search("iris").df()

Let us create 500 notebook objects with fake titles, save, and search them:

In [ ]:
transforms = [ln.Transform(name=title, type="notebook") for title in ln.core.datasets.fake_bio_notebook_titles(n=500)]
ln.save(transforms)

# search
ln.Transform.search("intestine").df().head(5)

```{note}

Currently, the LaminHub UI search is more powerful than the search of the `lamindb` open-source package.

```

## Leverage relations

Django has a double-under-score syntax to filter based on related tables.

This syntax enables you to traverse several layers of relations and leverage different comparators.

In [ ]:
ln.Artifact.filter(created_by__handle__startswith="testuse").df()  

The filter selects all artifacts based on the users who ran the generating notebook.

Under the hood, in the SQL database, it's joining the artifact table with the run and the user table.



## Comparators

You can qualify the type of comparison in a query by using a comparator.

Below follows a list of the most import, but Django supports about [two dozen field comparators](https://docs.djangoproject.com/en/stable/ref/models/querysets/#field-lookups) `field__comparator=value`.

### and

In [ ]:
ln.Artifact.filter(suffix=".jpg", created_by=user).df()

### less than/ greater than

Or subset to artifacts smaller than 10kB. Here, we can't use keyword arguments, but need an explicit where statement.

In [ ]:
ln.Artifact.filter(created_by=user, size__lt=1e4).df()

### in

In [ ]:
ln.Artifact.filter(suffix__in=[".jpg", ".fastq.gz"]).df()

### order by

In [ ]:
ln.Artifact.filter().order_by("-updated_at").df()

### contains

In [ ]:
ln.Transform.filter(name__contains="search").df().head(5)

And case-insensitive:

In [ ]:
ln.Transform.filter(name__icontains="Search").df().head(5)

### startswith

In [ ]:
ln.Transform.filter(name__startswith="Research").df()

### or

In [ ]:
ln.Artifact.filter(ln.Q(suffix=".jpg") | ln.Q(suffix=".fastq.gz")).df()

### negate/ unequal

In [ ]:
ln.Artifact.filter(~ln.Q(suffix=".jpg")).df()

Clean up the test instance.

In [ ]:
!rm -r ./test-registries
!lamin delete --force test-registries